This notebook contains the instructions for reproducing results presented in  "*Environmental and genealogical signals on DNA methylation in a widespread apomictic dandelion lineage*" by V.N. Ibañez, M. van Antro, C. Peña Ponton, S. Ivanovic, C.A.M. Wagemaker, F. Gawehns, K.J.F. Verhoeven.

## Load data and set R environment

In this section, we will load the dataset to run the script, configure the working directory and environment.

In [ ]:
#@title Load files
%load_ext rpy2.ipython
!rm -r *
!mkdir results rawData annotation scripts tmp

!wget -c -O scripts/commonFunctions.R https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/Rscripts/commonFunctions.R
!wget -c -O rawData/AseI-NsiI_Design_withPlotInfos.txt https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/AseI-NsiI_Design_withPlotInfos.txt
!wget -c -O rawData/Csp6I-NsiI_Design_withPlotInfos.txt https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/Csp6I-NsiI_Design_withPlotInfos.txt

!wget -c -O rawData/AseI-NsiI_methylation.bed https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/AseI-NsiI_petite_methylation.bed
!wget -c -O rawData/Csp6I-NsiI_methylation.bed https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/Csp6I-NsiI_petite_methylation.bed

!wget -c -O annotation/Csp6I-NsiI_mergedAnnot.csv https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/Csp6I-NsiI_mergedAnnot.csv
!wget -c -O annotation/AseI-NsiI_mergedAnnot.csv https://raw.githubusercontent.com/VeronicaNoe/epiTree/main/data4r/AseI-NsiI_mergedAnnot.csv


In [ ]:
%%R
#@title Set R environment
rm(list=ls())
wd<-getwd()
baseDir <- gsub("/results", "", wd)
scriptDir <- file.path(baseDir, "scripts")


In [ ]:
%%R
#@title Install R packages
install.packages(c("data.table","plyr","dplyr", "reshape2", "tidyverse", "ggplot2"), quiet=TRUE)


In [ ]:
%%R
#@title Load packages silently
suppressPackageStartupMessages({
  library("data.table")
  library("plyr") #ddply
  library("dplyr") #select
  library("reshape2") #melt
  library("tidyverse") #mutate
  library("ggplot2")
  source(file.path(scriptDir, "commonFunctions.R"), local=TRUE)
})

# Filtering data step-by-step

In this section, we will explore chunk of code to filter the one dataset: *AseI-NsiI*


## Load and explore data

In [ ]:
#@title
%%R
RE<-"AseI-NsiI"
designTable <- file.path(paste0(baseDir, "/rawData/",RE, "_Design_withPlotInfos.txt"))
infileName <- file.path(paste0(baseDir,"/rawData/",RE,"_methylation.bed"))
annotationFile <- file.path(paste0(baseDir, "/annotation/",RE, "_mergedAnnot.csv"))
rDir <- file.path(baseDir)
myData <- f.load.methylation.bed(infileName) 
colNamesForGrouping <- c("Treat")
sampleTab <- f.read.sampleTable(designTable, colNamesForGrouping) # see commonFunctions.R
totalCols <- grep("_total$", colnames(myData), value = TRUE)
methCov <- myData[,totalCols]
colnames(methCov) <- gsub("_total$", "", colnames(methCov))
#Number of Citosynes (C) in original data set. 
dimA<-dim(myData)[1]
# Match to remove bad samples
commonSamples <- sort(intersect(colnames(methCov), rownames(sampleTab)))
#allSamples <- union(colnames(methCov), rownames(sampleTab))
samplesToRemove <- setdiff(colnames(methCov), commonSamples) #order mathers! biggest first
if (length(samplesToRemove) > 0) {
  f.print.message("Removing", length(samplesToRemove), "samples!")
  cat(paste0(samplesToRemove, collapse = '\n'), '\n')
}
### remove bad samples
sampleTab <- sampleTab[commonSamples,]
dim(methCov)
methCov <- methCov[, commonSamples]
myData <- myData[, c("chr", "pos", "context", "samples_called", paste0(rep(commonSamples, each = 2), c("_methylated", "_total")))]

===  2022 Sep 17 10:13:11 AM === removing 503 positions with multiple contexts. 
===  2022 Sep 17 10:13:11 AM === Removing 0 samples due to the sampleRemovalInfo column 


In [ ]:
#@title
%%R
print(myData[1:10,1:10]) 

   chr pos context samples_called sample_40_AseI_methylated
1    1   8     CHH             21                        NA
2    1  10     CHH             21                        NA
3    1  14     CHH             21                        NA
4    1  17     CHH              9                        NA
5    1  20     CHH             21                        NA
6    1  23     CHG              9                        NA
7    1  25     CHG             21                        NA
8    1  28     CHH             21                        NA
9    1  30     CHH             21                        NA
10   1  32     CHH             21                        NA
   sample_40_AseI_total sample_30_AseI_methylated sample_30_AseI_total
1                    NA                         0                    3
2                    NA                         0                    3
3                    NA                         0                    3
4                    NA                        NA       

In [ ]:
#@title
%%R
cat(paste0("Number of Citosynes (C) in original data set: ", dimA))

Number of Citosynes (C) in original data set: 8986

## Filtering data by minimum read coverage in each C's.

The threshold is 10X.

In [ ]:
#@title
%%R
bins<-c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.9999, 1)
quaCov<-quantile(methCov, bins, na.rm=TRUE) # to detect high coverage reads
  
#transform in NA all values below 10X
minCov<-10
methCov[(methCov < minCov)] <- NA
quaCovB<-quantile(methCov, bins, na.rm=TRUE) # to detect high coverage reads
  
#set number of samples that are the 80% of samples
minCountPerGroup <- round((length(commonSamples)/2)*0.8, 0)
tabForSummary <- data.frame(sample = rownames(sampleTab), group = sampleTab$group, stringsAsFactors = FALSE)
callsPerGroup <- f.summarize.columns(!is.na(methCov), tabForSummary, sum)
maskToKeep <- rowSums(callsPerGroup >= minCountPerGroup) == ncol(callsPerGroup) # c's that are in at least 80% samples

if (sum(maskToKeep) == 0) {
  f.print.message("No cytosine passed the filter, no output.")
  quit("no", 0)
}

#remove C's that arenot in 80% samples
myData <- myData[maskToKeep,]
  
#subset and store
dimB<-dim(myData)[1]
dimB

## Filtering data by maximum read coverage in each C's.

The threshold is the value in the top distribution.

In [ ]:
#@title
%%R
totalCols <- grep("_total$", colnames(myData), value = TRUE)
methCov <- myData[,totalCols]
colnames(methCov) <- gsub("_total$", "", colnames(methCov))

maxCov<-as.numeric(quaCovB[11]) #99.99%
methCov[(methCov > maxCov)] <- NA
minCountPerGroup <- round((length(commonSamples)/2)*0.8, 0)

tabForSummary <- data.frame(sample = rownames(sampleTab), group = sampleTab$group, stringsAsFactors = FALSE) #summarize data for each sample
callsPerGroup <- f.summarize.columns(!is.na(methCov), tabForSummary, sum)
maskToKeep <- rowSums(callsPerGroup >= minCountPerGroup) == ncol(callsPerGroup) # c's that are in at least 80% samples

if (sum(maskToKeep) == 0) {
  f.print.message("No cytosine passed the filter, no output.")
  #quit("no", 0)
}

# subset and store
myData <- myData[maskToKeep,]

## Save the filtered dataset



In [ ]:
#@title
%%R
#C's in final data
totalCols <- grep("_total$", colnames(myData), value = TRUE)
methCov <- myData[,totalCols]
methCov[(methCov < minCov) | (methCov > maxCov)] <- NA
dimC<-dim(myData)[1]
quaCovC<-quantile(methCov, bins, na.rm=TRUE)
length(unique(myData$chr)) 


In [ ]:
#@title Check final coverage
%%R
#see quantile for each sample
qua4samples<-c()
for (s in 1:ncol(methCov)){
  qDis<-quantile(methCov[,1], na.rm=TRUE)
  sampleDis<-c(colnames(methCov)[s], qDis)
  qua4samples<-rbind(qua4samples, sampleDis)
}
ctxt<-c("CG", "CHG", "CHH")
tmp <- myData[,c("context",totalCols)]
qDisTotal<-quantile(tmp[,2:dim(tmp)[2]], na.rm=TRUE)
totalMean<-melt(tmp[,2:dim(tmp)[2]],)
totalKeep<-mean(totalMean[,2], na.rm=TRUE)
qDisTotal<-c(data.frame(qDisTotal)[,1], totalKeep)
out<-c()
for(c in ctxt){
  tmp2<-subset(tmp, context==c)
  toMean<-melt(tmp2[,2:dim(tmp2)[2]],)
  toKeep<-mean(toMean[,2], na.rm=TRUE)
  qDis<-quantile(tmp2[,2:dim(tmp2)[2]], na.rm=TRUE)
  count<-data.frame(qDis)[,1]
  tmp_out<-c(count, toKeep)
  out<-cbind(out,tmp_out)
}
out<-cbind(out,qDisTotal)
rownames(out)<-c(rownames(data.frame(qDis)),"mean")
colnames(out)<-c(ctxt,"qDisTotal")
write.table(out, file = paste0(baseDir,"/results/",RE,"_qDis.txt"), sep = '\t', col.names = TRUE, row.names = TRUE, quote = FALSE)

In [ ]:
#@title Save the report
%%R
# Save reports
sink(paste0(baseDir,"/tmp/", RE,"_Filtering_report.txt"))
paste0("Number of Cs from bed file: ", dimA)
paste0("Quantile coverage from bed file: ")
quaCov
paste0("Number of Cs after 10x filtering: ", dimB)
paste0("Quantile coverage after 10X: ")
quaCovB
paste0("Number of Cs final file: ", dimC)
paste0("Quantile coverage final file: ")
quaCovC
sink()

# Run filtering step

In this section, the code will run the previous steps for both datasets: *AseI-NsiI* and *Csp6I-NsiI*

In [ ]:
%%R
#@ title Filter both data set: AseI-NsiI and Csp6I-NsiI
RE<-c("AseI-NsiI", "Csp6I-NsiI")
for (r in 1:length(RE)){
  designTable <- file.path(paste0(baseDir, "/rawData/",RE[r], "_Design_withPlotInfos.txt"))
  infileName <- file.path(paste0(baseDir,"/rawData/",RE[r],"_methylation.bed"))
  annotationFile <- file.path(paste0(baseDir, "/annotation/",RE[r], "_mergedAnnot.csv"))
  rDir <- file.path(baseDir)
  
  ## Load and explore data
  myData <- f.load.methylation.bed(infileName) 
  colNamesForGrouping <- c("Treat")
  sampleTab <- f.read.sampleTable(designTable, colNamesForGrouping) # see commonFunctions.R
  print(str(myData))
  totalCols <- grep("_total$", colnames(myData), value = TRUE)
  methCov <- myData[,totalCols]
  colnames(methCov) <- gsub("_total$", "", colnames(methCov))
  
  #Number of Citosynes (C) in original data set. 
  dimA<-dim(myData)[1]
  
  ## Match to remove bad samples
  commonSamples <- sort(intersect(colnames(methCov), rownames(sampleTab)))
  #allSamples <- union(colnames(methCov), rownames(sampleTab))
  samplesToRemove <- setdiff(colnames(methCov), commonSamples) #order mathers! biggest first
  if (length(samplesToRemove) > 0) {
    f.print.message("Removing", length(samplesToRemove), "samples!")
    cat(paste0(samplesToRemove, collapse = '\n'), '\n')
  }
  ### remove bad samples
  sampleTab <- sampleTab[commonSamples,]
  dim(methCov)
  methCov <- methCov[, commonSamples]
  myData <- myData[, c("chr", "pos", "context", "samples_called", paste0(rep(commonSamples, each = 2), c("_methylated", "_total")))]

  ## Filtering data by minimum and maximum coverage
  #First start with minimum coverage
  bins<-c(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.9999, 1)
  quaCov<-quantile(methCov, bins, na.rm=TRUE) # to detect high coverage reads
  
  #transform in NA all values below 10X
  minCov<-10
  methCov[(methCov < minCov)] <- NA
  quaCovB<-quantile(methCov, bins, na.rm=TRUE) # to detect high coverage reads
  
  #set number of samples that are the 80% of samples
  minCountPerGroup <- round((length(commonSamples)/2)*0.8, 0)
  tabForSummary <- data.frame(sample = rownames(sampleTab), group = sampleTab$group, stringsAsFactors = FALSE)
  callsPerGroup <- f.summarize.columns(!is.na(methCov), tabForSummary, sum)
  maskToKeep <- rowSums(callsPerGroup >= minCountPerGroup) == ncol(callsPerGroup) # c's that are in at least 80% samples

  if (sum(maskToKeep) == 0) {
    f.print.message("No cytosine passed the filter, no output.")
    quit("no", 0)
  }

  #remove C's that arenot in 80% samples
  myData <- myData[maskToKeep,]
  
  #subset and store
  dimB<-dim(myData)[1]
  dimB

  #Second, set maximum filtering after removing minimum filtering
  totalCols <- grep("_total$", colnames(myData), value = TRUE)
  methCov <- myData[,totalCols]
  colnames(methCov) <- gsub("_total$", "", colnames(methCov))

  maxCov<-as.numeric(quaCovB[11]) #99.99%
  methCov[(methCov > maxCov)] <- NA
  minCountPerGroup <- round((length(commonSamples)/2)*0.8, 0)

  tabForSummary <- data.frame(sample = rownames(sampleTab), group = sampleTab$group, stringsAsFactors = FALSE) #summarize data for each sample
  callsPerGroup <- f.summarize.columns(!is.na(methCov), tabForSummary, sum)
  maskToKeep <- rowSums(callsPerGroup >= minCountPerGroup) == ncol(callsPerGroup) # c's that are in at least 80% samples

  if (sum(maskToKeep) == 0) {
    f.print.message("No cytosine passed the filter, no output.")
    #quit("no", 0)
  }

  # subset and store
  myData <- myData[maskToKeep,]
  write.table(myData, file = paste0(baseDir,"/results/",RE[r],"_methylation.filtMETH"), sep = '\t', col.names = TRUE, row.names = FALSE, quote = FALSE)
  
  #C's in final data
  totalCols <- grep("_total$", colnames(myData), value = TRUE)
  methCov <- myData[,totalCols]
  methCov[(methCov < minCov) | (methCov > maxCov)] <- NA
  dimC<-dim(myData)[1]
  quaCovC<-quantile(methCov, bins, na.rm=TRUE)
  length(unique(myData$chr)) 

  #see quantile for each sample
  qua4samples<-c()
  for (s in 1:ncol(methCov)){
    qDis<-quantile(methCov[,1], na.rm=TRUE)
    sampleDis<-c(colnames(methCov)[s], qDis)
    qua4samples<-rbind(qua4samples, sampleDis)
  }

  # Save reports
  sink(paste0(baseDir,"/tmp/", RE[r],"_Filtering_report.txt"))
  paste0("Number of Cs from bed file: ", dimA)
  paste0("Quantile coverage from bed file: ")
  quaCov
  paste0("Number of Cs after 10x filtering: ", dimB)
  paste0("Quantile coverage after 10X: ")
  quaCovB
  paste0("Number of Cs final file: ", dimC)
  paste0("Quantile coverage final file: ")
  quaCovC
  sink()
  # check coverage
  ctxt<-c("CG", "CHG", "CHH")
  tmp <- myData[,c("context",totalCols)]
  qDisTotal<-quantile(tmp[,2:dim(tmp)[2]], na.rm=TRUE)
  totalMean<-melt(tmp[,2:dim(tmp)[2]],)
  totalKeep<-mean(totalMean[,2], na.rm=TRUE)
  qDisTotal<-c(data.frame(qDisTotal)[,1], totalKeep)
  out<-c()
  for(c in ctxt){
    tmp2<-subset(tmp, context==c)
    toMean<-melt(tmp2[,2:dim(tmp2)[2]],)
    toKeep<-mean(toMean[,2], na.rm=TRUE)
    qDis<-quantile(tmp2[,2:dim(tmp2)[2]], na.rm=TRUE)
    count<-data.frame(qDis)[,1]
    tmp_out<-c(count, toKeep)
    out<-cbind(out,tmp_out)
  }
  out<-cbind(out,qDisTotal)
  rownames(out)<-c(rownames(data.frame(qDis)),"mean")
  colnames(out)<-c(ctxt,"qDisTotal")
  write.table(out, file = paste0(baseDir,"/results/",RE[r],"_qDis.txt"), sep = '\t', col.names = TRUE, row.names = TRUE, quote = FALSE)
}

===  2022 Sep 17 09:47:11 AM === removing 503 positions with multiple contexts. 
===  2022 Sep 17 09:47:11 AM === Removing 0 samples due to the sampleRemovalInfo column 
'data.frame':	8986 obs. of  84 variables:
 $ chr                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ pos                      : int  8 10 14 17 20 23 25 28 30 32 ...
 $ context                  : chr  "CHH" "CHH" "CHH" "CHH" ...
 $ samples_called           : num  21 21 21 9 21 9 21 21 21 21 ...
 $ sample_40_AseI_methylated: int  NA NA NA NA NA NA NA NA NA NA ...
 $ sample_40_AseI_total     : int  NA NA NA NA NA NA NA NA NA NA ...
 $ sample_30_AseI_methylated: int  0 0 0 NA 0 NA 2 1 0 0 ...
 $ sample_30_AseI_total     : int  3 3 3 NA 3 NA 3 3 3 3 ...
 $ sample_37_AseI_methylated: int  1 0 0 0 0 1 1 0 0 0 ...
 $ sample_37_AseI_total     : int  2 2 2 1 2 1 2 2 2 2 ...
 $ sample_20_AseI_methylated: int  NA NA NA NA NA NA NA NA NA NA ...
 $ sample_20_AseI_total     : int  NA NA NA NA NA NA NA NA NA NA ...
 $ sample_29_As